<a href="https://colab.research.google.com/github/pokem1402/section_project_one/blob/main/EDA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading Dataset

 EDA_1.ipynb 을 통해 Year, Publisher, Genre의 결손치를 보충하고 결손된 데이터를 제거한 데이터를 불러온다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/My Drive/dataset/vgames2_modified.csv").drop(axis=1, columns="Unnamed: 0")

# 2. 분석할 데이터와 분류 값들을 분리

1. 게임 이름
2. 게임 플랫폼
3. 게임 출시년도
4. 게임 장르
5. 게임 퍼블리셔

6. 게임의 각 지역의 판매량 (북미, 유럽, 일본, 그외 지역)

In [3]:
_name = df.Name
_platform = df.Platform
_year = df.Year
_genre = df.Genre
_publisher = df.Publisher

In [4]:
df_numeric = df.take(range(5,9), axis=1)

In [9]:
try:
  df_numeric.dtype= df_numeric.astype(float)
except BaseException as e:
  print(e)


could not convert string to float: '480K'


# 3. 데이터에 표기 오류값 정정

바로 윗 코드에서 string을 float로 변환하지 못하는 문제가 발생했다. 어떤 항목에서 문제를 일으키는 것인지 확인해보자.

#### 우선 float로 변환 가능한 데이터 row와 불가능한 row를 분리한다.

In [6]:
# 변환 가능한 row들의 집합
valid_index = df_numeric[(df_numeric.replace("[.]", "", regex=True).applymap(lambda x : x.isdigit())).all(1)].index
# 변환 불가능한 row들의 집합
invalid_index = df_numeric[(df_numeric.replace("[.]", "", regex=True).applymap(lambda x : x.isdigit()) == False).any(1)].index
# 두 집합이 서로 독립이면서 합집합이 원래 집합이 되는 것을 확인한다.
df.shape[0] == valid_index.shape[0] + invalid_index.shape[0]

True

In [7]:
# Data check
# 우선 우세적인 수치데이터의 최대값과 최소값을 알아보자.
data = df_numeric.replace("K", 0, regex=True).replace("M", 0, regex=True).loc[invalid_index]
data.dtype=data.astype(float)


data["NA_Sales"].max()
# print(data.columns)
# for column in data.columns:
#   print(column)
# data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


TypeError: ignored

In [ ]:
df.loc[invalid_index]

데이터를 들여다볼 때 K와 M 이 존재하는데 이는 $10^3$을 나타내는 접두어인 Kilo와 $10^6$을 나타내는 접두어인 Mega일 가능성이 높다.

 따라서 k를 1000, 그리고 M을 1000000으로 교체하자. 그리고 기존에 존재하는 값이 어떤 지표인지 모르기 때문에 비교하기 위하여 일단 음수값으로 둔다.

In [ ]:
df_numeric.loc[invalid_index].applymap(lambda x : x.replace("K", 000, regex=True))